# Projekt 6 - Lázmérő készülék

A COVID járvány időszakában nem nehéz amiatt aggódni, hogy vajon elkaptuk-e a vírust vagy sem. Egyik nap arra ébredtél, hogy furcsán érzed magad, mintha kicsit melegebb is lenne a homlokod, fáradtabb is vagy. Első lépésként gondoltad megméred a lázad, de nem találtál lázmérőt. 

Mint egy ügyes kis tech alkotó, (maker) azonnal eszedbe jutott, hogy van neked thermisztorod, összedobhatsz gyorsan abból is egy lázmérőt. Mint az igazi lázmérőknél, egy gomb lenyomása után csak várunk, amíg a mérő kiírja a hőmérsékletünket.

## Mit fogsz készíteni?

Egy NTCC 10k nevű thermisztorból, egy MCP3008 ADC-ből, egy 10 kOhm ellenállásból és egy nyomógombból álló áramkört fogsz összerakni egy feszültségosztó elrendezésben. Ebben a projektben egy gomb lenyomásával elkezded rögzíteni az aktuális hőmérsékletet 0.8 másodpercenként. Akkor állítod le a mérést, ha a három utolsó mérés mindegyike közötti különbség egyenként kisebb mint 0.01 $^\circ$C.

## Mit tanulsz meg?

A lázmérő szenzor elkészítésével a következőket tanulod meg:

* Hogyan kommunikáljunk egy ADC-vel.
* Hogyan használjunk gombokat.
* Hogyan használjunk ```if - else``` szerkezetet.
* Hogyan kell fájlokat beolvasni az ```open``` használatával.
* Hogyan importálunk egy függvényt egy fájból amit mi írtunk.
* Hogyan interpolálunk ismeretlen értékekre. 
* Hogyan számoljuk a thermisztor ellenállását egy feszültségosztóval.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Importálni az MCP3008, LED objektumokat, ```numpy``` csomagot, függvényt az interpolálásra és fájl beolvasó függvényt.
* Inicializálni a gombot és az ADC-t.
* Definiálni egy függvényt, ami egy megadott fájlból kinyeri a thermisztor hőmérséklet-ellenállás függését. 
* Definiálunk egy függvényt ami a mért feszültséget átalakítja ellenállássá a feszültségosztó elve alapján.
* Definiálunk egy függvényt, hogy a hőmérséklet-ellenállás függés ismeretlen pontjait interpoláljuk.
* Definiálunk egy összehasonlító függvényt ami egyenként összehasonlítja az utolsó három mérés értékét.
* A nyomógombbal elindítani a mérést.
* Egy végtelen ```while``` ciklusban mérjük a feszültséget, kiszámoljuk az ellenállás változást, abból az aktuális hőmérsékletet, és a hőmérsékleteket és megállapítjuk az összehasonlító függvénnyel, hogy mikor van vége a mérésnek.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) NTCC 10k szenzor: [itt vásárolhatsz](https://www.tme.eu/hu/details/ntcc-10k/tht-ntc-merotermisztorok/sr-passives/)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) Ellenállás: [itt vásárolhatsz](https://www.tme.eu/hu/katalog/metal-film-tht-ellenallasok_112313/?s_order=desc&search=ellenallas&s_field=1000011)

e) MCP3008 I/P ADC: [itt vásárolhatsz](https://www.tme.eu/hu/details/mcp3008-i_p/a-d-konverterek-ic-k/microchip-technology/)

f) Nyomó gomb: [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) vagy [építhetsz]()

## A kapcsolási rajz

<img src="schema/prog05_schema.png" width=800 height=600 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t. Az áramkör részletes összekötési magyarázata a [Thermisztorok]() című bevezető leírásban található. **N.B. Ha nem a lent megadott módon kötjük be az ADC-t, akkor figyelni kell arra, hogy a szoftverben a lábak definiálásánál ezt jelezni kell, illetve tudunk kell, hogy akkor nem hardwares SPI kommunikáció lesz.**

0) Kössük sorba a NTCC 10k thermisztort és a 10 kOhm ellenállást. A thermisztor üres lábára kapcsoljuk a Raspberry Pi 3.3 V-os tápját, míg az ellenállás üres lábára a földelést. A thermisztor és az ellenállás csatlakozását pedig kössük az MCP3008 CH7-es csatornájára.

1) Kössük össze a Raspberry Pi egyik földelését az MCP3008 AGND ás DGND lábaival (fekete drót).

2) Kössük az MCP3008 *VDD* és *VREF* nevű lábait a Raspberry Pi 3.3 V-os tüskéjére. 

3) Kössük az MCP3008 *CLK* nevű lábát a Raspberry Pi *GPIO11* tüskéjére. 

4) Kössük az MCP3008 *DOUT* nevű lábát a Raspberry Pi *GPIO09* tüskéjére. 

5) Kössük az MCP3008 *DIN* nevű lábát a Raspberry Pi *GPIO10* tüskéjére.

6) Kössük az MCP3008 *CS* nevű lábát a Raspberry Pi *GPIO08* tüskéjére.

7) A nyomógomb egyik lábát a földelésre a másikat pedig a *GPIO02*-es tüskére kötjük.

## A kód

Nyissunk meg egy új python fájlt és mentsük el pl. ```ntcc_temperature_button.py``` név alatt. A ```gpiozero``` csomagnak nincs beépített objektuma ami általánosan a thermisztorokkat tudná kezelni, így minden egyes thermisztorhoz nekünk kell megoldani a szoftveres kommunikációt az adott eszközzel.

### A thermisztor tesztelése

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, a hőmérséklet kiolvasására.

Mivel a thermisztor analóg jelet ad ki, amit a Raspberry Pi nem tud értelmezni, közbe iktatünk egy analóg digitális átalakítót, MCP3008. Ez a chip SPI protokolon keresztül kommunikál és a ```gpiozero``` csomagnak van hozzá objektuma, amit a ```Button``` klasszokkal együtt beimportálunk. 

A konvertert először inicializáljuk (a dokumentációban több példa is szerepel, hogy miképp lehet azt megtenni). Bemenő paraméterként megadjuk, hogy a thermisztoros feszültségosztóból kiolvasott feszültséget a 7-es csatornára csatlakoztattuk. Teszteléshez elég meghívni az objektum ```voltage``` metódusát, amit kiírathatunk a képernyőre.

```ntcc_temperature_button.py```:

In [2]:
from gpiozero import MCP3008, Button

mcp = MCP3008(channel=7)

while True:
	print(mcp.voltage)              

### Nyomógombok tesztelése

Következő lépésben definiálunk egy gombot, amit aa *GPIO02*-re kötünk.  Ahhoz, hogy leteszteljük helyesen kötöttük-e be a gombot és, hogy működik, írunk egy segédfüggvényt ```test()```, amit a gomb lenyomásakor meghívunk majd. A függvény meghívásakor egyszerűen csak kiíratjuk a képernyőre, hogy a gombot megnyomtuk. Végül ezt a függvényt hozzá kell rendelnünk a gombhoz, méghozzá most a gomb lenyomásának pillanatához a ```when_pressed``` metódushoz.

```ntcc_temperature_button.py```:

In [ ]:
from gpiozero import MCP3008, Button

mcp = MCP3008(channel=7)
button_start = Button(17)

def test():
    print('Button is pressed')
    
button_start.when_pressed = test

while True:
    pass

A kód végén még elindítunk egy végtelen ciklust, ami nem csinál semmit, így csak a gombok lenyomása lesz az egyetlen egy esemény amit a számítógép jelezhet. 

Mentsük le a kódot és futtasuk le a pythonnal. Teszteljük, hogy a gombok lenyomására megjelenik-e az üzenet a képernyőn. 

### Régi függvények újrahasznosítása

Mivel a projektünk ugyanazon az áramkörön alapszik, mint az előző, így annak függvényeit újrahasznosíthatjuk. A ```read_temp_raw``` függvényt csak beimportáljuk a ```temperature_functions``` modulból (ügyeljünk arra, hogy a modul ugyanabban a mappában legyen mint a mi aktuális fájlunk), míg a ```read_mapping```, ```voltage2resistance``` és a ```interpolate_temperature``` függvényeket csak újra bemásoljuk a végső kódba.

### Függvény az utolsó 3 mérés összehasonlítására

Az egyszerűség kedvéért a hőmérsékletünk megállapításához összehasonlítjuk az utolsó 3 mérés értékét egyenként egymással és ha azok nem térnek el jobban mint egy $\epsilon$ kis érték, akkor az átlaguk lesz a hőmérsékletünk.

Ehhez definiálunk egy ```compare(tc, eps=0.01)``` nevű függvényt, aminek a bemenő paraméterei a 3 elemből álló hőmérséklet lista ```tc``` és egy általunk választható kicsi szám ```eps=0.01```, ami megmondja, hogy mekkora egymás utáni hőmérsékletkülönbség esetén van meg a végső hőmérséklet. 

Egy ```if``` szerkezetben megvizsgáljuk, hogy a 3 mért hőmérséklet különbség abszolút értéke egyenként kisebb-e mint az $\epsilon$ értéke (0.01 $^\circ$C), azaz konvergál-e a mérésünk, ```if abs(tc[0]-tc[1]) < eps and abs(tc[0]-tc[2]) < eps and abs(tc[2]-tc[1]) < eps:```. Ha igen, akkor vesszük a 3 mérés átlagát, ```t_mean = sum(tc)/len(tc)```, kiírjuk annak eredményét a képernyőre és végül kilépünk a pythonból, ```sys.exit(0)```. Ha nem, akkor semmi nem történik és kilépünk a függvényből, várunk az újabb meghívásra. 

In [1]:
def compare(tc, eps=0.01):
	if abs(tc[0]-tc[1]) < eps and abs(tc[0]-tc[2]) < eps and abs(tc[2]-tc[1]) < eps:
		t_mean = sum(tc)/len(tc)
		print(f'Your body temperature is: {t_mean:.2f} Celsius degree')
		sys.exit(0)

### A teljes kód

Már csak az eddig leírtak összetákolása van hátra kiegészítve egy kis mérési utasítással. Kommentelt sorok jelzik az új elemeket a kódban.

A python csomagok és modulok beimportálása és a segéd függvények definiálása után, elsőként beolvassuk a hőmérséklet-ellenállás táblázatot, ```tc, resistance = read_mapping('ntcc.csv')```. Inicializálunk egy üres listát, amiben a mért hőmérsékleteket fogjuk tárolni. Kiíratunk egy üzenetet a képernyőre, hogy a mérés előtt tegyük a kezünket a thermisztorra, majd nyomjuk meg a gombot ami elindítja a mérést, ```button.wait_for_press()```.

Ezután elindítunk egy végtelen ```while``` ciklust, amiben először bevezetünk egy 0.8 másodperces késleltetést a mérések között, majd kiolvassuk a mért feszültséget a thermisztoron, és az alapján meghatározzuk a thermisztor ellenállásának értékét, ```rt = voltage2resistance(mcp.voltage, v_in=3.3, r=9.82e3)```. Ha meg van az ellenállás, a táblázatból interpolálással megkapjuk a hozzá tartozó hőmérsékletet, ```t_therm = interpolate_temperature(rt, tc, resistance)```. 

Miután megkaptuk a mért hőmérsékletet, beletesszük a tároló listába, ```t_body.append(t_therm)```. A következő lépésben megvizsgáljuk, hogy a tárolónknak van-e már 3 eleme, ```if len(t_body) < 3:```. Ha nincs, akkor indítunk egy újabb mérést. Ha van, akkor összehasonlítjuk őket, hogy közelít-e már az értékük, ```compare(t_body, eps=0.01)```, majd az összehasonlítás után kitöröljük a tároló első elemét, így biztosítva, hogy mindig csak 3 mérést hasonlítsunk össze. 

```ntcc_temperature_button.py```:

In [ ]:
from gpiozero import MCP3008, Button
import numpy as np                 # numpy importalas
import sys,time                    # sys es time importalas
from scipy.interpolate import interp1d   # interpolalas importalasa
from temperature_functions import read_temp_raw


mcp = MCP3008(channel=7)
button = Button(2)


def read_mapping(name):
	data = read_temp_raw(name)
	data.pop(0)
	tc = []
	r = []
	for line in data:
		clean_data = line.strip().split(',')
		tc.append(float(clean_data[0]))
		r.append(float(clean_data[1])*1000)
	return np.array(tc), np.array(r)
	
def voltage2resistance(v_out, v_in=3.3, r=9.82e3):
	return r * v_out / (v_in - v_out)

def interpolate_temperature(resistance, tc, r):
	f = interp1d(r,tc)
	return f(resistance)

def compare(tc, eps=0.01):
	if abs(tc[0]-tc[1]) < eps and abs(tc[0]-tc[2]) < eps and abs(tc[2]-tc[1]) < eps:
		t_mean = sum(tc)/len(tc)
		print(f'Your body temperature is: {t_mean:.2f} Celsius degree')
		sys.exit(0)

# homerseklet-ellenallas tablazat beolvasasa
tc, resistance = read_mapping('ntcc.csv')		
t_body = []                     # homerseklet tarolo inicializalas

print('Place your hand on the thermistor')   #  tegyuk a kezunk a thermisztorra
button.wait_for_press()  # varj a gomb lenyomasaig


while True:           # vegtelen ciklus inditasa
	time.sleep(0.8)   # 0.8s idokesleltetes ket meres kozott
    # mert feszultsegbol az ellenallas kiszamitasa
	rt = voltage2resistance(mcp.voltage, v_in=3.3, r=9.82e3)
    # homersklet meghatarozasa az ellenallas ismereteben
	t_therm = interpolate_temperature(rt, tc, resistance)
    # tarolohoz uj homersekletet adni
	t_body.append(t_therm)
	
	if len(t_body) < 3: # ha a tarolo hossza kisebb mint harom
		continue        # legyen uj meres 
	else:               # ha nem kisebb mint 3
        # hasonlitsa ossze a 3 meres erteket
		compare(t_body, eps=0.01)
        # torolje a tarolobol az elso mert erteket
		t_body.pop(0)

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```ntcc_temperature_button.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```ntcc_temperature_button.py```-t elmentettük. Ott begépelve a ```python ntcc_temperature_button.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a program elindításával megjelenik egy felszólítás a képernyőn, hogy fogjuk meg a thermisztort majd nyomjuk meg a gombot. Ezután a képernyőre kiíródik a mért hőmérséklet, mindaddig, amíg az jelentősen változik.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Írjunk egy plusz feltételt a ```while``` ciklusba, hogy ha az nem áll meg 15 másodpercen belül, akkor az épp az aktuális utolsó 3 mérés átlaga legyen a hőmérséklet.
* A ```t_body``` változót írjuk át ```numpy.array``` formátumúvá, és ennek megfelelően módosítsuk a kód többi részét, beleértve a ```compare``` függvényt is.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) https://tutorials-raspberrypi.com/mcp3008-read-out-analog-signals-on-the-raspberry-pi/

2) MCP3008 - https://gpiozero.readthedocs.io/en/stable/api_spi.html#mcp3008

3) Gomb objektum leírása - https://gpiozero.readthedocs.io/en/stable/recipes.html#button

4) if-else leírás - https://www.programiz.com/python-programming/if-elif-else

5) modul importálás - https://www.programiz.com/python-programming/modules

6) interpolálás - https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html

7) numpy array használata - https://numpy.org/doc/stable/reference/generated/numpy.array.html

8) string műveletek - https://www.programiz.com/python-programming/string